In [41]:
from datasets import load_dataset
import torch
from torch import nn
import numpy as np

#ds = load_dataset("scene_parse_150", split="train[:30]")
ds = load_dataset("cifar10", split="train[:30]")

Generating train split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset cifar10 downloaded and prepared to /home/arefhz75/.cache/huggingface/datasets/cifar10/plain_text/1.0.0/447d6ec4733dddd1ce3bb577c7166b986eaa4c538dcd9e805ba61f35674a9de4. Subsequent calls will reuse this data.


In [42]:
ds = ds.train_test_split(test_size=0.3)
train_ds = ds["train"]
test_ds = ds["test"]

In [43]:
import json
from huggingface_hub import cached_download, hf_hub_url

repo_id = "huggingface/label-files"
filename = "ade20k-hf-doc-builder.json"
#id2label = json.load(open(cached_download(hf_hub_url(repo_id, filename, repo_type="dataset")), "r"))
id2label = {k: str(k) for k in range(180)}
label2id = {str(k): k for k in range(180)}
num_labels = len(id2label)

In [118]:
from transformers import AutoImageProcessor

feature_extractor = AutoImageProcessor.from_pretrained("nvidia/mit-b0", reduce_labels=True, do_resize=False)

loading configuration file preprocessor_config.json from cache at /home/arefhz75/.cache/huggingface/hub/models--nvidia--mit-b0/snapshots/ed0b85c75627eab6a3c6989627450cf95f115381/preprocessor_config.json
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
{param_name} should be a dictionary on of the following set of keys: {VALID_SIZE_DICT_KEYS}, got {size}. Converted to {size_dict}.
Image processor SegformerImageProcessor {
  "do_normalize": true,
  "do_reduce_labels": false,
  "do_rescale": true,
  "do_resize": false,
  "image_mean": [
    0.485,
    0.456,
    0.406
  ],
  "image_processor_type": "SegformerImageProcessor",
  "image_std": [
    0.229,
    0.224,
    0.225
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 512,
    "width": 512
  }
}



In [119]:
from torchvision.transforms import ColorJitter

jitter = ColorJitter(brightness=0.25, contrast=0.25, saturation=0.25, hue=0.1)

In [194]:
def hot_one(x):
    ret = np.zeros((1,10))
    ret[0,x] = 1
    return ret
def train_transforms(example_batch):
    #images = [jitter(x) for x in example_batch["image"]]
    #print("batch   ", example_batch)
    images = [x for x in example_batch["img"]]
    #print("im  ",images)
    #labels = [np.array(hot_one(x)) for x in example_batch["label"]]
    labels = np.array([x for x in example_batch["label"]])
    #labels = [x for x in example_batch["annotation"]]
    #print("lb  ",labels)
    inputs = feature_extractor(images, labels)
    #inputs = feature_extractor(images)
    return inputs


def val_transforms(example_batch):
#     images = [x for x in example_batch["image"]]
#     labels = [x for x in example_batch["annotation"]]
    images = [x for x in example_batch["img"]]
    labels = np.array([x for x in example_batch["label"]])
    inputs = feature_extractor(images, labels)
    return inputs
train_ds.set_transform(train_transforms)
test_ds.set_transform(val_transforms)

In [195]:
import evaluate

metric = evaluate.load("mean_iou")

In [196]:
def compute_metrics(eval_pred):
    with torch.no_grad():
        logits, labels = eval_pred
        logits_tensor = torch.from_numpy(logits)
        logits_tensor = torch.nn.functional.interpolate(
            logits_tensor,
            size=labels.shape[-2:],
            mode="bilinear",
            align_corners=False,
        ).argmax(dim=1)

        pred_labels = logits_tensor.detach().cpu().numpy()
        metrics = metric.compute(
            predictions=pred_labels,
            references=labels,
            num_labels=num_labels,
            ignore_index=255,
            reduce_labels=False,
        )
        for key, value in metrics.items():
            if type(value) is np.ndarray:
                metrics[key] = value.tolist()
        return metrics

In [197]:
from transformers import AutoModelForSemanticSegmentation, TrainingArguments, Trainer

pretrained_model_name = "nvidia/mit-b0"
nn_model = AutoModelForSemanticSegmentation.from_pretrained(
    pretrained_model_name, id2label=id2label, label2id=label2id
)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self. nn_model = nn_model
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(5760, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, **kwargs):
        #print("x   ", kwargs)
        x = self. nn_model(pixel_values=kwargs['pixel_values'])
        logits = x.logits
        output = torch.sigmoid(logits)[0]
        #output = np.transpose(output, (1,2,0))
        output = output.view(2,-1)
        #print("out   ",output)
        #print("out   ",output.shape)
        logits = self.linear_relu_stack(output)
        #print("logits   ", logits.shape)
        #print("logits   ", logits)
        #logits = logits.argmax(axis=1)
        logits = torch.softmax(logits,dim=-1)
        #print("logits   ", logits.shape)
        #print("logits   ", logits)
        return torch.mean(logits), _
    
device = "cuda" if torch.cuda.is_available() else "cpu"
model = NeuralNetwork().to(device)
print(model)

loading configuration file config.json from cache at /home/arefhz75/.cache/huggingface/hub/models--nvidia--mit-b0/snapshots/ed0b85c75627eab6a3c6989627450cf95f115381/config.json
Model config SegformerConfig {
  "_name_or_path": "nvidia/mit-b0",
  "architectures": [
    "SegformerForImageClassification"
  ],
  "attention_probs_dropout_prob": 0.0,
  "classifier_dropout_prob": 0.1,
  "decoder_hidden_size": 256,
  "depths": [
    2,
    2,
    2,
    2
  ],
  "downsampling_rates": [
    1,
    4,
    8,
    16
  ],
  "drop_path_rate": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_sizes": [
    32,
    64,
    160,
    256
  ],
  "id2label": {
    "0": "0",
    "1": "1",
    "2": "2",
    "3": "3",
    "4": "4",
    "5": "5",
    "6": "6",
    "7": "7",
    "8": "8",
    "9": "9",
    "10": "10",
    "11": "11",
    "12": "12",
    "13": "13",
    "14": "14",
    "15": "15",
    "16": "16",
    "17": "17",
    "18": "18",
    "19": "19",
    "20": "20",
    "21": "21",

NeuralNetwork(
  (nn_model): SegformerForSemanticSegmentation(
    (segformer): SegformerModel(
      (encoder): SegformerEncoder(
        (patch_embeddings): ModuleList(
          (0): SegformerOverlapPatchEmbeddings(
            (proj): Conv2d(3, 32, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3))
            (layer_norm): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
          )
          (1): SegformerOverlapPatchEmbeddings(
            (proj): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
            (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
          )
          (2): SegformerOverlapPatchEmbeddings(
            (proj): Conv2d(64, 160, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
            (layer_norm): LayerNorm((160,), eps=1e-05, elementwise_affine=True)
          )
          (3): SegformerOverlapPatchEmbeddings(
            (proj): Conv2d(160, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
            (laye

In [199]:
training_args = TrainingArguments(
    output_dir="segformer-b0-scene-parse-150",
    learning_rate=6e-5,
    num_train_epochs=5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    save_total_limit=3,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_steps=20,
    eval_steps=5,
    logging_steps=1,
    eval_accumulation_steps=5,
    remove_unused_columns=False,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    #compute_metrics=compute_metrics
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running training *****
  Num examples = 21
  Num Epochs = 5
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 55
  Number of trainable parameters = 6977822


Step,Training Loss,Validation Loss
5,0.100000,No log
10,0.100000,No log
15,0.100000,No log
20,0.100000,No log
25,0.100000,No log
30,0.100000,No log
35,0.100000,No log
40,0.100000,No log
45,0.100000,No log
50,0.100000,No log


***** Running Evaluation *****
  Num examples = 9
  Batch size = 2
***** Running Evaluation *****
  Num examples = 9
  Batch size = 2
***** Running Evaluation *****
  Num examples = 9
  Batch size = 2
***** Running Evaluation *****
  Num examples = 9
  Batch size = 2
Saving model checkpoint to segformer-b0-scene-parse-150/checkpoint-20
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Deleting older checkpoint [segformer-b0-scene-parse-150/checkpoint-60] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 9
  Batch size = 2
***** Running Evaluation *****
  Num examples = 9
  Batch size = 2
***** Running Evaluation *****
  Num examples = 9
  Batch size = 2
***** Running Evaluation *****
  Num examples = 9
  Batch size = 2
Saving model checkpoint to segformer-b0-scene-parse-150/checkpoint-40
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Deleting older checkpoint [segformer-b0-scene-parse-150/checkpoint-80] due to args.s

TrainOutput(global_step=55, training_loss=0.10000000094825572, metrics={'train_runtime': 8.8475, 'train_samples_per_second': 11.868, 'train_steps_per_second': 6.216, 'total_flos': 0.0, 'train_loss': 0.10000000094825572, 'epoch': 5.0})